## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

With `link_only`, only between-dataset record comparisons are generated. No within-dataset record comparisons are created, meaning that the model does not attempt to find within-dataset duplicates.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/duckdb/link_only.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [2]:
from splink.datasets import splink_datasets

df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
241,241,Freya,NaN,1970-12-17,London,f.s@flynn.com,62
278,278,Taylor,Lola,2016-10-23,Aberdeen,lolat86@bishop-gilels.com,71


In [3]:
from splink.linker import Linker
from splink.database_api import DuckDBAPI
from splink.blocking_rule_library import block_on
import splink.comparison_library as cl
import splink.comparison_template_library as ctl


settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "comparisons": [
        ctl.NameComparison(
            "first_name",
        ),
        ctl.NameComparison("surname"),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            invalid_dates_as_null=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
}

linker = Linker(
    [df_l, df_r],
    settings,
    database_api=DuckDBAPI(),
    input_table_aliases=["df_left", "df_right"],
)

In [4]:
linker.completeness_chart(cols=["first_name", "surname", "dob", "city", "email"])

alt.LayerChart(...)

In [5]:
from splink.blocking_rule_library import block_on, BlockingRuleCreator

deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email"),
]

# TODO: estimate_probability_two_random_records_match should accept
# BlockingRuleCreator objects
deterministic_rules = [
    br.get_blocking_rule("duckdb") if isinstance(br, BlockingRuleCreator) else br
    for br in deterministic_rules
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.0033.
This means that amongst all possible pairwise record comparisons, one in 303.29 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 824.29 matching pairs


In [6]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [7]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)
session_first_name = linker.estimate_parameters_using_expectation_maximisation(
    block_on("first_name")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.408 in the m_probability of email, level `Exact match on email`
Iteration 2: Largest change in params was 0.109 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0279 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.00844 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.003 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00115 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.000449 in probability_two_random_reco

In [8]:
results = linker.predict(threshold_match_probability=0.9)

In [9]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,3.240182,0.904298,df_left,df_right,297,296,None,David,-1,Banks,...,1987-06-12,1986-06-12,4,Birimnham,BirmBaghnm,0,david.banks26@thomas-bell.com,None,-1,1
1,3.282285,0.906794,df_left,df_right,746,745,oeL,Leo,0,Bradley,...,1993-03-27,1992-03-26,1,None,Reaidng,-1,leo.bradley54@green.net,leo.bradley54@green.net,3,1
2,3.480409,0.917771,df_left,df_right,908,909,Isabelle,None,-1,Carter,...,1997-11-26,1998-12-26,1,Bradford,Bradford,1,isabellec@hill-williams.com,None,-1,1
3,3.816989,0.933748,df_left,df_right,685,684,Rosie,Rosie,3,Johnoston,...,1978-11-23,1979-12-23,1,Sheffield,None,-1,rosiej32@robinson-moran.net,None,-1,0
4,4.085891,0.944387,df_left,df_right,207,209,Eloilt,Elltiot,1,Page,...,1979-05-25,1979-04-23,2,None,London,-1,e.page13@dyer-williams.com,None,-1,1
